In [1]:
from urllib import request
from urllib import parse
import json

In [2]:
import sqlite3

conn = sqlite3.connect('../db/BusStation2.db')
cur = conn.cursor()
cur.execute("SELECT DISTINCT(busID) FROM BusStation2")

In [3]:
low_bus = []

for lb in cur.fetchall():
    low_bus.append(int(lb[0]))

In [4]:
conn2 = sqlite3.connect('../db/elevator.db')
cur2 = conn2.cursor()
cur2.execute("SELECT ~~ FROM ~~")

OperationalError: near "FROM": syntax error

In [5]:
def fullroute(sx, sy, ex, ey):   #이용자가 입력한 출발, 목적지
    url = "https://api.odsay.com/v1/api/searchPubTransPath"
    params = {
    'SX' : sx,
    'SY':  sy,
    'EX' : ex,
    'EY' : ey,
    'OPT' :2,
    'apiKey' : "ttfS2WdLF4rG/hfj+d20/Lcxp6TIyVHnnW/IgBBHh60"
    }
    
    paramUrl = parse.urlencode(params)
    paramBytes = paramUrl.encode("utf-8")

    req = request.Request(url, data=paramBytes)
    request.get_method = lambda: 'GET'
    res = request.urlopen(req)
    result = res.read().decode("utf-8") 
    resultObj = json.loads(result)
    
    #경로가 존재
    try:  
        pathList = resultObj['result']['path']
        
        #보행약자 전용 경로로 수정
        better_path = []
#         low_bus = [1244]   #lowbus List

        for path in pathList:
            for sub in path['subPath']:
                better = True

                if sub['trafficType']==1:  #지하철
                    print(sub['startName'])
                    if 'startExitNo' in sub.keys():
                        print(sub['startExitNo']+"번출구", sub['startExitX'], sub['startExitY'])
                        #승강기 DB와 비교해서 해당 출구에 승강기가 없으면(or 사용불가능) 다른 출구로 update
                        #승강기와 연결된 출구가 없다면 better=False, break
                    print(sub['endName'])
                    if 'endExitNo' in sub.keys():
                        print(sub['endExitNo']+"번출구")
                        #승강기 DB와 비교해서 해당 출구에 승강기가 없으면 다른 출구로 update
                        #승강기와 연결된 출구가 없다면 better=False, break
                    print()

                elif sub['trafficType'] ==2:  #버스
                    temp_lane = []
                    for l in sub['lane']:
                        if l['busID'] in low_bus:
                            print(l['busID'])
                            temp_lane.append(l)
                    #sub['lane'] = temp_lane
                    if not temp_lane:
                        better = False
                        break

            if better:
                better_path.append(path)
        
        return [better_path, (sx, sy, ex, ey)]
    
    except:
        if resultObj['error'][0]['code']==-98:
            print("출, 도착지가 700m이내입니다.")
            return (sx, sy, ex, ey)
        else:
            print(resultObj['error'])

In [34]:
from bs4 import BeautifulSoup

class Pedestrian:
    #TMap API 호출 결과 반환
    def __init__(self, sx, sy, ex, ey):
        self.pos= [sx, sy, ex, ey]
        
        tmapURL = "https://api2.sktelecom.com/tmap/routes/pedestrian?version=1&format=xml"
        data = {
            'searchOption': 30,
            'startX' :sx ,
            'startY' :  sy, 
            'endX' : ex,
            'endY' :  ey,
            'reqCoordType' : "WGS84GEO",
            'resCoordType' : "EPSG3857",
            'angle' : "172",
            'startName': "출발지",
            'endName': "도착지",
        }

        paramUrl = parse.urlencode(data)
        paramBytes = paramUrl.encode("utf-8")

        self.req = request.Request(tmapURL, data=paramBytes, headers={'appKey':"b9e1fdd0-0495-4f88-8b28-3e1ae2e84b19", 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'})
        request.get_method = lambda: 'GET'
        res = request.urlopen(self.req)
        self.result = res.read().decode("utf-8") 
        self.xml = BeautifulSoup(self.result, "html.parser")
        try:
            self.totaltime =  int(self.xml.find("tmap:totaltime").text)//60+1
            self.totaldistance = int(self.xml.find("tmap:totaldistance").text)
        except:
            print(self.xml.find("message"))
        
    

In [7]:
class Subway:
    def __init__(self, on, way, code, num, off, tm, desc):
        self.start = on
        self.way = way
        self.code = code
        self.num_station = num
        self.end = off
        self.sectionTm = tm
        self.desc = desc
    

In [8]:
class Bus:
    def __init__(self, on, code, num, off, tm, desc):
        self.start = on
        self.code = code
        self.num_station = num
        self.end = off
        self.sectionTm = tm
        self.desc = desc

In [9]:
def eachroute(better_path):
    if type(better_path) == list:
        XY = better_path[1]
        better_path = better_path[0]

        splitpath = []

        for path in better_path:
            onepath = {'totaltime': 0, 'score': 30, 'pathType': 0, 'trans': 0, 'desc': "", 'totalwalk': 0}

            subPath = path['subPath']

            idx = 0

            for index in range(len(subPath)):
                idx += 1

                if subPath[index]['trafficType'] == 1:
                    on = subPath[index]['startName']
                    way = subPath[index]['way']
                    code = subPath[index]['lane'][0]['subwayCode']
                    num = subPath[index]['stationCount']
                    off = subPath[index]['endName']
                    tm = subPath[index]['sectionTime']
                    desc = "{}역에서 {}방향 {}호선 승차, {}정거장 후 {}역에서 하차 - 약 {}분 이동".format(on, way, code, num, off, tm)
                    onepath['desc'] += "{}역에서 {}호선 승차.  ".format(on, code)
                    onepath[idx] = Subway(on, way, code, num, off, tm, desc)
                    onepath['totaltime'] += tm
                    onepath['pathType'] = 1 if onepath['pathType'] in [0, 1] else 3

                elif subPath[index]['trafficType'] == 2:
                    on = subPath[index]['startName']
                    code = ""
                    for i in range(len(subPath[index]['lane'])):
                        if i == (len(subPath[index]['lane']) - 1):
                            code += str(subPath[index]['lane'][i]['busNo'])
                        else:
                            code += str(subPath[index]['lane'][i]['busNo']) + ", "
                    num = subPath[index]['stationCount']
                    off = subPath[index]['endName']
                    tm = subPath[index]['sectionTime']
                    desc = "<{}>에서 {}번 버스 승차, {}정거장 후 <{}>에서 하차 - 약 {}분 이동".format(on, code, num, off, tm)
                    onepath['desc'] += "<{}>에서 {}번 버스 승차.  ".format(on, code)
                    onepath[idx] = Bus(on, code, num, off, tm, desc)
                    onepath['totaltime'] += tm
                    onepath['pathType'] = 2 if onepath['pathType'] in [0, 2] else 3

                elif subPath[index]['trafficType'] == 3:
                    if index == 0:
                        # TMap에서 출발지부터 최초 정류장or지하철역 출구까지의 보행 경로 안내
                        if subPath[1]['trafficType'] == 1:
                            startX = subPath[1]['startExitX']
                            startY = subPath[1]['startExitY']
                        elif subPath[1]['trafficType'] == 2:
                            startX = subPath[1]['startX']
                            startY = subPath[1]['startY']
                        onepath[idx] = Pedestrian(XY[0], XY[1], startX, startY)
                        onepath['totaltime'] += onepath[idx].totaltime
                        onepath['totalwalk'] += onepath[idx].totaltime

                    elif index == len(subPath) - 1:
                        # TMap에서 마지막 정류장 or 지하철역 출구부터의 도착지까지의 보행 경로 안내
                        if subPath[index - 1]['trafficType'] == 1:
                            endX = subPath[index - 1]['endExitX']
                            endY = subPath[index - 1]['endExitY']
                        elif subPath[index - 1]['trafficType'] == 2:
                            endX = subPath[index - 1]['endX']
                            endY = subPath[index - 1]['endY']
                        onepath[idx] = Pedestrian(endX, endY, XY[2], XY[3])
                        onepath['totaltime'] += onepath[idx].totaltime
                        onepath['totalwalk'] += onepath[idx].totaltime

                    else:
                        #                         onepath['pass'] = []

                        if subPath[index - 1]['trafficType'] == 1 and subPath[index + 1][
                            'trafficType'] == 1:  # 지하철 - 지하철 환승
                            station = subPath[index - 1]['endName']
                            fromlane = subPath[index - 1]['lane'][0]['subwayCode']
                            tolane = subPath[index + 1]['lane'][0]['subwayCode']
                            # print(station+"역의 " +str(fromlane)+"호선에서 "+str(tolane)+"호선으로 환승")
                            # 해당 지하철역의 from호선 -> to호선까지 환승시에 걸리는 시간만큼 subPath[index]['sectionTime'] 증가
                            #                             onepath[idx] = "{}역의 {}호선에서 {}호선으로 환승".format(station, fromlane, tolane)
                            sectionTm = subPath[index]['sectionTime'] + 0
                            onepath[idx] = {'station': station, 'fromlane': fromlane, 'tolane': tolane,
                                            'sectionTm': sectionTm}
                            onepath['totaltime'] += sectionTm
                            onepath['totalwalk'] += sectionTm




                        elif subPath[index - 1]['trafficType'] == 2 and subPath[index + 1][
                            'trafficType'] == 2:  # 버스 - 버스 환승
                            startX = subPath[index - 1]['endX']
                            startY = subPath[index - 1]['endY']
                            endX = subPath[index + 1]['startX']
                            endY = subPath[index + 1]['startY']

                            onepath[idx] = Pedestrian(startX, startY, endX, endY)
                            onepath['totaltime'] += onepath[idx].totaltime
                            onepath['totalwalk'] += onepath[idx].totaltime

                        elif subPath[index - 1]['trafficType'] == 1 and subPath[index + 1][
                            'trafficType'] == 2:  # 지하철 - 버스 환승
                            startX = subPath[index - 1]['endExitX']
                            startY = subPath[index - 1]['endExitY']
                            endX = subPath[index + 1]['startX']
                            endY = subPath[index + 1]['startY']
                            onepath[idx] = Pedestrian(startX, startY, endX, endY)
                            onepath['totaltime'] += onepath[idx].totaltime
                            onepath['totalwalk'] += onepath[idx].totaltime

                        elif subPath[index - 1]['trafficType'] == 1 and subPath[index + 1][
                            'trafficType'] == 2:  # 버스 - 지하철 환승
                            startX = subPath[index - 1]['endX']
                            startY = subPath[index - 1]['endY']
                            endX = subPath[index + 1]['startExitX']
                            endY = subPath[index + 1]['startExitY']
                            onepath[idx] = Pedestrian(startX, startY, endX, endY)
                            onepath['totaltime'] += onepath[idx].totaltime
                            onepath['totalwalk'] += onepath[idx].totaltime

                        onepath['score'] -= 5  # 환승시 편의 지수 하락
                        onepath['trans'] += 1

            splitpath.append(onepath)

    elif type(better_path) == tuple:
        splitpath = [Pedestrian(*better_path)]

    return splitpath


In [19]:
full[1]

(126.9027279, 37.5349277, 126.922725, 37.547914)

In [37]:
full = fullroute(126.9027279,37.5349277,126.922725,37.547914)

당산
4번출구 126.903165671079 37.5351929282617
합정

합정
상수
1번출구

당산
4번출구 126.903165671079 37.5351929282617
합정
7번출구

1244
1812
1197
1228
합정
8번출구 126.912857037513 37.5493662587383
상수
1번출구

1166
합정
8번출구 126.912857037513 37.5493662587383
상수
1번출구

당산
4번출구 126.903165671079 37.5351929282617
합정
4번출구

989
당산
4번출구 126.903165671079 37.5351929282617
합정
8번출구

1166
1168
618
618
1168
1154
1279


In [41]:
len(full[0])

5

In [42]:
if len(full[0]) > 7:
    full[0] = full[0][:7]
pathList = eachroute(full)

In [45]:
pathList[1]

{'totaltime': 21,
 'score': 25,
 'pathType': 3,
 'trans': 1,
 'desc': '당산역에서 2호선 승차.  <합정역>에서 7011, 7013A, 7013B번 버스 승차.  ',
 'totalwalk': 5,
 1: <__main__.Pedestrian at 0x24c25705ba8>,
 2: <__main__.Subway at 0x24c256ea278>,
 3: <__main__.Pedestrian at 0x24c256ea358>,
 4: <__main__.Bus at 0x24c25746470>,
 5: <__main__.Pedestrian at 0x24c25746160>}

In [70]:
pathList[6][1].totaltime

4.916666666666667

In [66]:
pathList[6][1].pos[0], pathList[6][1].pos[1],pathList[6][1].pos[2],pathList[6][1].pos[3]

(126.9027279, 37.5349277, 126.9008027, 37.5346106)

In [94]:
pathList

[{'totaltime': 17,
  'score': 25,
  'pathType': 1,
  'trans': 1,
  'desc': '당산역에서 2호선 승차.&nbsp합정역에서 6호선 승차.&nbsp',
  'totalwalk': 0,
  1: <__main__.Pedestrian at 0x23f0e7dc9b0>,
  2: <__main__.Subway at 0x23f0dc94240>,
  3: {'station': '합정', 'fromlane': 2, 'tolane': 6, 'sectionTm': 0},
  4: <__main__.Subway at 0x23f0dc943c8>,
  5: <__main__.Pedestrian at 0x23f0dc949e8>},
 {'totaltime': 18,
  'score': 25,
  'pathType': 3,
  'trans': 1,
  'desc': '당산역에서 2호선 승차.&nbsp<합정역>에서 7011, 7013A, 7013B번 버스 승차. &nbsp',
  'totalwalk': 2,
  1: <__main__.Pedestrian at 0x23f0e7ea940>,
  2: <__main__.Subway at 0x23f0e7f8dd8>,
  3: <__main__.Pedestrian at 0x23f0e7f8f28>,
  4: <__main__.Bus at 0x23f0e81b3c8>,
  5: <__main__.Pedestrian at 0x23f0e81b1d0>},
 {'totaltime': 31,
  'score': 25,
  'pathType': 3,
  'trans': 1,
  'desc': '<양화대교전망카페>에서 5712, 6716, 6712번 버스 승차. &nbsp합정역에서 6호선 승차.&nbsp',
  'totalwalk': 15,
  1: <__main__.Pedestrian at 0x23f0e827908>,
  2: <__main__.Bus at 0x23f0e84b6a0>,
  4: <__main__

In [77]:
p = pathList[0]

In [87]:
for i in p:
    print(p[i])

17
25
1
1
당산역에서 2호선 승차.&nbsp합정역에서 6호선 승차.&nbsp
0
{'station': '합정', 'fromlane': 2, 'tolane': 6, 'sectionTm': 0}


## 웹 구성
#### 메인화면
- 출발지, 목적지, 도착시간 입력, 화면상에 지도 출력

#### 경로결과화면
- 모든 경로와 각 경로의 통합 정보(총 소요시간, 환승횟수, 이동편의지수 등)를 추천/지하철만/버스만/버스+지하철 에 따라 리스트로 출력

#### 세부경로화면
- 선택한 경로의 세부 경로들의 리스트를 세부 경로 타입에 맞게 출력(도보or지하철or버스)

### flask routing

In [ ]:
@app.route('/fullpath')
def fullpath(stX, stY, eX, eY, departTm):
    #이용자가 입력하는 출발, 목적지 좌표 및 시간
    
    pathList= eachroute(fullroute(stX, stY, eX, eY))  #subPath + 메타데이터를 dict으로 담고있는 경로들의 list
    #각 경로마다의 subPath를 객체로 저장 -> Pedestrian() , Subway(), Bus()
    #웹 구현 시 각 subPath의 객체타입에 따라 보여지는 형식이 다름
        
    
    return render_template("fullpath.html", pathList=pathList)  #모든 경로를 대중교통 타입에 따라 구분하여 화면에 보여주는 html

#     pathList = [(126.9027279 , 37.5349277, 126.922725, 37.547914),(startX , startY, endX, endY)]
#     return render_template('security/route.html', pathList=pathList)

In [ ]:
@app.route('/subpath')
def subpath(path):   #하나의 경로를 구성하는 subpath들을 보여줌
    
    return render_template("subpath.html", path = path)  #subpath들의 대중교통 타입에 따라 다른 형식으로 화면에 보여주는 html

In [86]:
#jinja에서 type 함수를 사용하기 위한 셋팅

import jinja2

env=jinja2.Environment()
env.filters['type'] = type

template =env.from_string("""begin {{ note|type == t }} end""")
# print(template.render(note=A(), t=A))
print(template.render(note=1, t=int))


begin True end


In [ ]:
#임시 코드
fullpath_html = """
<body>
    {% for path in pathList %}
        hassubway = false
        hasbus = false
        
        {% if pathType ==1 %}]
             <div class="subwayonly">
                    <p>경로의 총 소요시간, 환승횟수, 이동편의지수</p>
                </div>
        {%elif pathType==2 %}
             ...
        {%else%}
            ...
            
        {%if score > 1 0%}
            <div class="recommend"> ... </div>
        {% endif %}
    {% endfor %}
</body>
"""

In [2]:
#임시 코드
subpath_html = """
<body>
    {% for subpath in path %}
        {% if a.type == Subway or a.type==Bus%}
            <p>탭 안에 지하철이나 버스 정보 text로 표현</p>
        {% elif {% if a.type == Pedetrian %}
            <script TMap>
                initTmap({{ subpath.sx,  subpath.sy,  subpath.ex, subpath.ey}});
            </script>
            <div id="map_div"></div>
            <p id="result" or "route"></p>
        {% endif %}
    {% endfor %}
    
</body>"""